# Lab 9 - Summarizing Attendance and Practice Attempts

In the last lab, you combined the information for attendance and practice quizzes into one combined table.  In this lab, you will transform these data into two summary tables.

1. For each class, you will create a table that summarizes the attendance of all students in that class.
2. For each class that has practice quizzes, you will create a table that summarized the number of attempts, overall and for each module.

In [1]:
%reset -f
import pandas as pd
from dfply import *
from glob import glob
import re 

In [2]:
df_init = pd.read_csv("./data/lab8_attendance_final.csv")
df_init.sample(7)

,Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent,Class,Category,Module
681,19846151,wf4016jq,Scotsman,McCluskey,2,20,20,2019-03-01 11:33:00,2019-03-01 11:38:00,100 %,stat491s1,Practice,4.0
2517,14819010,kx3806uj,Ross,Galbreath,3,16,20,2019-03-01 11:40:00,2019-03-01 11:46:00,80 %,dsci494s7,Practice,3.0
872,15135961,wd8670of,McKinley,Sabina,23,6,20,2019-03-01 11:01:00,2019-03-01 11:07:00,30 %,stat491s1,Practice,1.0
2846,17802189,ee7746ti,Ben,Aug,8,16,20,2019-03-01 11:12:00,2019-03-01 11:20:00,80 %,dsci494s7,Practice,1.0
1092,17802189,ee7746ti,Ben,Aug,9,1,1,2019-02-06 14:01:00,2019-02-06 14:07:00,100 %,dsci494s7,Attendance,NaN
285,13978323,tn9656sg,Appalachia,Leonardo,7,1,1,2019-02-01 15:02:00,2019-02-01 15:10:00,100 %,stat491s1,Attendance,NaN
257,12501099,af1090bg,Athenian,Acadia,12,1,1,2019-02-18 15:03:00,2019-02-18 15:10:00,100 %,stat491s1,Attendance,NaN


## Tasks

To complete this lab, complete each of the following tasks.

#### Task 1 - Attendance Summary

For each of the classes contained in `attendance_example.zip`, we want to create a table with three columns: `FirstName`, `LastName`, `Attendance`.  Note that `Attendance` will represent the maximum number of attempts on the quiz, which corresponds to that students number of days they have attended.  To complete this task, do the following.

1. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
2. Use a loop and your helper functions to create the attendance quiz table for each class and then write the contents to a csv file.

In [3]:
@dfpipe
def get_Name_Attendance(df):
    df = (df
          >>rename(Attempts = 'Attempt #')
          >>select(X.FirstName,X.LastName,X.Attempts,X.Class)
          >>group_by(X.FirstName,X.LastName,X.Class)
          >>summarise(Attendace = X.Attempts.max())
         )
    return df



@dfpipe
def filter_class(df,class_):
    df = df[df["Class"] == class_]
    return df

def write_class_csv(df,class_):
    df = (df
          >>get_Name_Attendance()
          >>filter_class(class_)
         )
    return df.to_csv("Data/Attendance_{}.csv".format(class_),index = True)

def write_all(df):
    class_ = set(df["Class"])
    for cls in class_:
        write_class_csv(df,cls)
    return None

write_all(df_init)






#### Task 2 - Practice Quiz Summary

Some of the classes contained in `attendance_example.zip` contain information about attempts on practice quizzes for four modules.  We want to create a table for each class that summarizes the practice quiz attempts.  This table should contain the following columns: `FirstName`, `LastName`, `Module 1`, `Module 2`, `Module 3`, `Module 4`, and `Total Attempts`.  Note that, for example, `Module 1` contains the total number of attempts each student made on the corresponding quiz and `Total Attempts` contains the total number of attempts on all four quizzes.


1. Create a list of class that have practice quizzes.
2. Write a helper function that takes a class identifier and the overall dataframe as arguments; and returns the table described above.
3. Use a loop and your helper functions to create the practice quiz table for each class and then write the contents to a csv file.

In [4]:
def get_class_with_quiz(df):
    class_with_quiz = [class_ for class_ in df.loc[df['Category'] == "Practice"]["Class"] ]
    return list(set(class_with_quiz))

@dfpipe
def filter_df_Quiz_class(df,class_):
    df = df.loc[df['Category'] == "Practice"]
    df = df.loc[df['Class']==class_]
    return df
@dfpipe
def fillNA(df):
    df = df.fillna(0)
    return df

@dfpipe
def get_table_quiz_DF(df,class_):
    
    df = (df
          >>rename(Attempts = 'Attempt #')
          >>select(X.FirstName,X.LastName,X.Module,X.Class,X.Category,X.Attempts)
          
         >>filter_df_Quiz_class(class_)
          >>group_by(X.FirstName,X.LastName,X.Module)
        >>summarise(score_max = X.Attempts.max())
         >> spread(X.Module,X.score_max)
          >>fillNA()
           >>rename(Module_1 = 2,Module_2 = 3,Module_3 = 4,Module_4 = 5)
          >>mutate(Total_attempts = X.Module_1+X.Module_2+X.Module_3+X.Module_4)
         )
    return df

def writer_csv(df,class_):
    df = df>>get_table_quiz_DF(class_)
    df.to_csv('./Data/lab9_{}_practice.csv'.format(class_))
    return None

def write_all_class(df):
    write = [writer_csv(df,cls) for cls in get_class_with_quiz(df)]
    return None
    

In [5]:
write_all_class(df_init)